In [ ]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame, permitiendo la detección de tipo más precisa
df = pd.read_csv('generadoras_activas_corregido.csv', low_memory=False)

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tslearn.preprocessing import TimeSeriesScalerMinMax
from scipy.cluster.hierarchy import linkage, fcluster
import itertools

def prediction_strength(train_labels, val_labels, n_clusters):
    """Calcula el Prediction Strength ajustado para tamaños diferentes de conjuntos"""
    def cluster_membership_matrix(labels, n_clusters):
        n_samples = len(labels)
        membership = np.zeros((n_samples, n_samples))

        for i in range(n_samples):
            for j in range(n_samples):
                if labels[i] == labels[j]:
                    membership[i, j] = 1
        return membership

    # Obtener matrices de pertenencia para el entrenamiento y la validación
    train_membership_matrix = cluster_membership_matrix(train_labels, n_clusters)
    val_membership_matrix = cluster_membership_matrix(val_labels, n_clusters)

    total_pairs = 0
    consistent_pairs = 0

    # Calcular consistencia de pares en entrenamiento y validación
    for i in range(min(len(train_labels), len(val_labels))):
        for j in range(i + 1, min(len(train_labels), len(val_labels))):
            if train_membership_matrix[i, j] == 1:
                total_pairs += 1
                if val_membership_matrix[i, j] == 1:
                    consistent_pairs += 1
            elif train_membership_matrix[i, j] == 0:
                total_pairs += 1
                if val_membership_matrix[i, j] == 0:
                    consistent_pairs += 1

    return consistent_pairs / total_pairs if total_pairs > 0 else 0

# Asegúrate de que las fechas en el DataFrame original estén en el formato datetime
df['FECHA'] = pd.to_datetime(df['FECHA'])

# Agrupar por fecha y llave, y sumar la columna 'TOTAL'
energia_por_fecha = df.groupby(['LLAVE NOMBRE', 'FECHA'])['TOTAL'].sum().reset_index()

# Pivotar los datos para tener fechas como columnas y llaves como filas
pivot_df = energia_por_fecha.pivot(index='LLAVE NOMBRE', columns='FECHA', values='TOTAL').fillna(0)

# Aplanar el MultiIndex después de la pivotación
pivot_df.columns = [f'{col.strftime("%Y-%m-%d")}' for col in pivot_df.columns]

# Añadir la columna de nombres de generadoras al DataFrame
pivot_df['LLAVE NOMBRE'] = pivot_df.index

# Convertir el DataFrame a un formato adecuado para el clustering
data = pivot_df.drop(columns=['LLAVE NOMBRE']).values

# Normalizar los datos con MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = scaler.fit_transform(data)

# Normalización específica de series temporales
data_normalized = TimeSeriesScalerMinMax().fit_transform(data_normalized.reshape((data_normalized.shape[0], data_normalized.shape[1], 1)))

# Convertir los datos normalizados en una matriz 2D para linkage
data_reshaped = data_normalized.reshape(data_normalized.shape[0], -1)

# Separar los datos en entrenamiento (90%) y validación (10%)
train_size = int(len(data_reshaped) * 0.9)
X_train, X_val = data_reshaped[:train_size], data_reshaped[train_size:]

# Probar con varios números de clústeres
results = []

# Métodos y rangos de número de clústeres
methods = ['ward', 'complete']  # Métodos de enlace solicitados
n_clusters_range = range(2, 31)  # De 2 a 6

# Probar combinaciones de métodos y números de clústeres
for method, n_clusters in itertools.product(methods, n_clusters_range):
    print(f"\nProbando con método: {method.upper()}, n_clusters={n_clusters}")

    # Iniciar cronómetro
    start_time = time.time()

    # Aplicar AHC con los métodos solicitados (ward y complete)
    linked = linkage(X_train, method=method)
    labels_train = fcluster(linked, t=n_clusters, criterion='maxclust')
    
    # Clustering para la validación
    linked_val = linkage(X_val, method=method)
    labels_val = fcluster(linked_val, t=n_clusters, criterion='maxclust')

    # Calcular el Prediction Strength
    ps_score = prediction_strength(labels_train, labels_val, n_clusters)

    # Detener el cronómetro
    end_time = time.time()
    execution_time = end_time - start_time

    # Guardar los resultados
    result = {
        'k': n_clusters,
        'Prediction Strength': ps_score,
        'Execution Time (s)': execution_time,
        'method': method
    }

    results.append(result)

    # Imprimir los resultados para cada combinación
    print(f"Tiempo de ejecución: {execution_time:.2f} segundos")
    print(f"Prediction Strength: {ps_score:.2f}\n")

# Mostrar los resultados finales
print("\nResultados finales:")
for result in results:
    print(f"k={result['k']}, method={result['method']} -> Prediction Strength: {result['Prediction Strength']:.2f}, Execution Time: {result['Execution Time (s)']:.2f} segundos")

# Convertir los resultados a DataFrame para graficar
results_df = pd.DataFrame(results)

# Graficar los resultados de Prediction Strength para cada método
plt.figure(figsize=(10, 6))
for method in methods:
    method_df = results_df[results_df['method'] == method]
    plt.plot(method_df['k'], method_df['Prediction Strength'], label=f'Prediction Strength ({method})', marker='o')

plt.xlabel('Número de Clusters (K)')
plt.ylabel('Prediction Strength')
plt.title('Prediction Strength para métodos Ward y Complete')
plt.legend()
plt.grid(True)
plt.show()

# Graficar los tiempos de ejecución para cada método
plt.figure(figsize=(10, 6))
for method in methods:
    method_df = results_df[results_df['method'] == method]
    plt.plot(method_df['k'], method_df['Execution Time (s)'], label=f'Tiempo de ejecución ({method})', marker='o')

plt.xlabel('Número de Clusters (K)')
plt.ylabel('Tiempo de ejecución (s)')
plt.title('Tiempo de ejecución para métodos Ward y Complete')
plt.legend()
plt.grid(True)
plt.show()


# WARD

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from tslearn.preprocessing import TimeSeriesScalerMinMax
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.metrics import pairwise_distances
import time

# Agrupar por fecha y llave, y sumar la columna 'TOTAL'
energia_por_fecha = df.groupby(['LLAVE NOMBRE', 'FECHA'])['TOTAL'].sum().reset_index()

# Pivotar los datos para tener fechas como columnas y llaves como filas
pivot_df = energia_por_fecha.pivot(index='LLAVE NOMBRE', columns='FECHA', values='TOTAL').fillna(0)

# Convertir las columnas de fecha a datetime
pivot_df.columns = pd.to_datetime(pivot_df.columns)

# Añadir la columna de nombres de generadoras al DataFrame
pivot_df['LLAVE NOMBRE'] = pivot_df.index

# Muestreo: seleccionar una fracción aleatoria de los datos (por ejemplo, el 10%)
sample_df = pivot_df.sample(frac=1, random_state=0)  # Ajusta la fracción según sea necesario

# Convertir el DataFrame muestreado a un formato adecuado
data = sample_df.drop(columns=['LLAVE NOMBRE']).values

# Normalización específica de series temporales
data_normalized = TimeSeriesScalerMinMax().fit_transform(data.reshape((data.shape[0], data.shape[1], 1)))

# Convertir los datos normalizados en una matriz 2D para linkage
data_reshaped = data_normalized.reshape(data_normalized.shape[0], -1)

# Empezar el cronómetro
start_time = time.time()

# Aplicar el algoritmo de agrupamiento jerárquico aglomerativo (AHC) con la métrica Ward
linked = linkage(data_reshaped, method='ward')

# Obtener etiquetas basadas en el número de clusters (por ejemplo, 6 clusters)
n_clusters = 10  # Ajusta el número de clusters según sea necesario
labels = fcluster(linked, t=n_clusters, criterion='maxclust')

# Añadir los resultados al DataFrame muestreado
sample_df['Cluster'] = labels

# Reordenar las columnas para mantener 'LLAVE NOMBRE' al principio
sample_df = sample_df[['LLAVE NOMBRE', 'Cluster'] + [col for col in sample_df.columns if col not in ['LLAVE NOMBRE', 'Cluster']]]

# Calcular la matriz de distancias euclidianas
distances = pairwise_distances(data_reshaped)

# Calcular el índice de Silueta usando la métrica euclidiana
silhouette_avg = silhouette_score(distances, labels, metric='euclidean')

# Detener el cronómetro
end_time = time.time()

# Calcular el tiempo de ejecución
execution_time = end_time - start_time

# Imprimir los resultados
print(f"Tiempo de ejecución: {execution_time:.2f} segundos")
print(f"Tiempo de ejecución: {execution_time / 60:.2f} minutos")
print(f"Índice de Silueta: {silhouette_avg:.2f}")

# Limpiar las columnas de `pivot_df` asegurándose de que sean fechas
pivot_df.columns = pd.to_datetime(pivot_df.columns, errors='coerce')

# Eliminar columnas que no se pudieron convertir a fechas
pivot_df = pivot_df.loc[:, pivot_df.columns.notna()]

# Obtener la lista de clusters únicos
clusters = sample_df['Cluster'].unique()

# Crear un DataFrame con la columna 'LLAVE NOMBRE' y 'TIPO' de df
tipo_df = df[['LLAVE NOMBRE', 'TIPO']].drop_duplicates()

# Crear un diccionario para almacenar los totales y proporciones de tipos por cluster
totales_cluster = {}
porcentaje_tipo_total = {}

# Inicializar un DataFrame para acumular el total de cada tipo
total_tipos_df = tipo_df['TIPO'].value_counts().to_frame(name='Total').reset_index()
total_tipos_df.columns = ['TIPO', 'Total']

# Iterar sobre cada cluster para calcular los totales y proporciones
for cluster in clusters:
    # Filtrar los datos del cluster
    cluster_data = sample_df[sample_df['Cluster'] == cluster]
    
    # Obtener la lista de generadoras en el cluster
    generadoras = cluster_data['LLAVE NOMBRE'].unique()
    
    # Obtener el tipo correspondiente para cada generadora
    tipos = tipo_df[tipo_df['LLAVE NOMBRE'].isin(generadoras)]
    
    # Contar las ocurrencias de cada tipo
    tipo_counts = tipos['TIPO'].value_counts()
    
    # Guardar el total de generadoras en el cluster
    total_cluster = tipo_counts.sum()
    totales_cluster[cluster] = total_cluster
    
    # Calcular el porcentaje de cada tipo dentro del cluster y agregar el conteo
    tipo_porcentaje_y_cantidad = tipo_counts.to_frame(name='Cantidad')
    tipo_porcentaje_y_cantidad['Porcentaje'] = (tipo_counts / total_cluster) * 100
    
    # Almacenar los porcentajes y cantidades en el diccionario
    porcentaje_tipo_total[cluster] = tipo_porcentaje_y_cantidad

# Mostrar los resultados totales por tipo y por cluster
print("Proporción total de cada tipo de generadora:")
print(total_tipos_df)
print()

print("Totales, cantidades y porcentajes de cada cluster:")
for cluster, total in totales_cluster.items():
    print(f"Cluster {cluster} - Total generadoras: {total}")
    print(f"Cantidad y porcentaje por tipo en el cluster {cluster}:")
    print(porcentaje_tipo_total[cluster])
    print()  # Línea en blanco para mayor legibilidad

# Calcular las series más cercanas al centroide para cada tipo dentro de cada cluster
for cluster in clusters:
    # Filtrar los datos del cluster
    cluster_data = sample_df[sample_df['Cluster'] == cluster]
    
    # Obtener la lista de generadoras en el cluster
    generadoras = cluster_data['LLAVE NOMBRE'].unique()
    
    # Filtrar las series temporales de las generadoras en el cluster
    cluster_series = pivot_df.loc[generadoras]
    
    # Obtener los tipos de energía en el cluster
    tipos_en_cluster = tipo_df[tipo_df['LLAVE NOMBRE'].isin(generadoras)]
    
    # Iterar sobre cada tipo de energía en el cluster
    for tipo in tipos_en_cluster['TIPO'].unique():
        # Filtrar las series temporales del tipo de energía actual
        series_tipo = cluster_series.loc[tipos_en_cluster[tipos_en_cluster['TIPO'] == tipo]['LLAVE NOMBRE']]
        
        if not series_tipo.empty:
            # Calcular el centroide (media) del tipo de energía
            centroide = series_tipo.mean(axis=0)
            
            # Calcular la distancia entre cada serie temporal y el centroide usando la distancia euclidiana
            distancias = np.array([np.linalg.norm(centroide.values - serie.values) for index, serie in series_tipo.iterrows()]).reshape(-1, 1)
            
            # Encontrar las 3 series más cercanas al centroide
            indices_mas_cercanas = np.argsort(distancias, axis=0)[:3].flatten()
            series_representativas = series_tipo.iloc[indices_mas_cercanas]
            
            # Graficar las 3 series temporales representativas
            fig, ax = plt.subplots(figsize=(12, 6))
            for j, (index, serie) in enumerate(series_representativas.iterrows()):
                ax.plot(pivot_df.columns, serie, label=f'Cluster {cluster} - Tipo {tipo} - Serie {j+1}', linestyle='-', marker='o')
            
            # Configuración del gráfico
            ax.set_title(f'Series Temporales Representativas del Cluster {cluster} - Tipo {tipo}')
            ax.set_xlabel('Fecha')
            ax.set_ylabel('Valor')
            ax.xaxis.set_major_locator(mdates.MonthLocator())
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
            ax.tick_params(axis='x', rotation=45)
            ax.legend(loc='upper right')

            plt.tight_layout()
            plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from tslearn.preprocessing import TimeSeriesScalerMinMax
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import time

# Agrupar por fecha y llave, y sumar la columna 'TOTAL'
energia_por_fecha = df.groupby(['LLAVE NOMBRE', 'FECHA'])['TOTAL'].sum().reset_index()

# Pivotar los datos para tener fechas como columnas y llaves como filas
pivot_df = energia_por_fecha.pivot(index='LLAVE NOMBRE', columns='FECHA', values='TOTAL').fillna(0)

# Convertir las columnas de fecha a datetime
pivot_df.columns = pd.to_datetime(pivot_df.columns)

# Añadir la columna de nombres de generadoras al DataFrame
pivot_df['LLAVE NOMBRE'] = pivot_df.index

# Muestreo: seleccionar una fracción aleatoria de los datos (por ejemplo, el 10%)
sample_df = pivot_df.sample(frac=1, random_state=0)

# Convertir el DataFrame muestreado a un formato adecuado
data = sample_df.drop(columns=['LLAVE NOMBRE']).values

# Normalización específica de series temporales
data_normalized = TimeSeriesScalerMinMax().fit_transform(data.reshape((data.shape[0], data.shape[1], 1)))
data_reshaped = data_normalized.reshape(data_normalized.shape[0], -1)

# Empezar el cronómetro
start_time = time.time()

# Aplicar el algoritmo de agrupamiento jerárquico aglomerativo (AHC) con la métrica Ward
linked = linkage(data_reshaped, method='ward')
n_clusters = 10
labels = fcluster(linked, t=n_clusters, criterion='maxclust')

# Detener el cronómetro
end_time = time.time()

# Calcular el tiempo de ejecución
execution_time = end_time - start_time

# Añadir los resultados al DataFrame muestreado
sample_df['Cluster'] = labels

# Reordenar las columnas para mantener 'LLAVE NOMBRE' al principio
sample_df = sample_df[['LLAVE NOMBRE', 'Cluster'] + [col for col in sample_df.columns if col not in ['LLAVE NOMBRE', 'Cluster']]]

# Visualización de los clústeres usando PCA
plt.figure(figsize=(10, 6))
for cluster in np.unique(labels):
    plt.scatter(data_pca[labels == cluster, 0], data_pca[labels == cluster, 1], label=f'Cluster {cluster}')
plt.title("Visualización de Clústeres usando PCA")
plt.xlabel("Componente Principal 1 (Varianza Máxima)")
plt.ylabel("Componente Principal 2 (Segunda Mayor Varianza)")
plt.legend()
plt.show()

# Visualización de los clústeres usando t-SNE
plt.figure(figsize=(10, 6))
for cluster in np.unique(labels):
    plt.scatter(data_tsne[labels == cluster, 0], data_tsne[labels == cluster, 1], label=f'Cluster {cluster}')
plt.title("Visualización de Clústeres usando t-SNE")
plt.xlabel("Componente t-SNE 1")
plt.ylabel("Componente t-SNE 2")
plt.legend()
plt.show()

# Calcular el índice de Silueta usando la métrica euclidiana
distances = pairwise_distances(data_reshaped)
silhouette_avg = silhouette_score(distances, labels, metric='euclidean')
print(f"Índice de Silueta: {silhouette_avg:.2f}")

# Mostrar los resultados de totales y porcentajes por tipo de generadora en cada clúster
tipo_df = df[['LLAVE NOMBRE', 'TIPO']].drop_duplicates()
clusters = sample_df['Cluster'].unique()
totales_cluster = {}
porcentaje_tipo_total = {}

for cluster in clusters:
    cluster_data = sample_df[sample_df['Cluster'] == cluster]
    generadoras = cluster_data['LLAVE NOMBRE'].unique()
    tipos = tipo_df[tipo_df['LLAVE NOMBRE'].isin(generadoras)]
    tipo_counts = tipos['TIPO'].value_counts()
    total_cluster = tipo_counts.sum()
    totales_cluster[cluster] = total_cluster
    tipo_porcentaje_y_cantidad = tipo_counts.to_frame(name='Cantidad')
    tipo_porcentaje_y_cantidad['Porcentaje'] = (tipo_counts / total_cluster) * 100
    porcentaje_tipo_total[cluster] = tipo_porcentaje_y_cantidad

print("Totales y porcentajes por tipo de generadora en cada clúster:")
for cluster, total in totales_cluster.items():
    print(f"Cluster {cluster} - Total generadoras: {total}")
    print(f"Cantidad y porcentaje por tipo en el cluster {cluster}:")
    print(porcentaje_tipo_total[cluster])
    print()

# Imprimir el tiempo de ejecución
print(f"Tiempo de ejecución: {execution_time:.2f} segundos")
print(f"Tiempo de ejecución: {execution_time / 60:.2f} minutos")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tslearn.preprocessing import TimeSeriesScalerMinMax
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.metrics import silhouette_score
import time

# Asegúrate de que las fechas en el DataFrame original estén en el formato datetime
df['FECHA'] = pd.to_datetime(df['FECHA'])

# Agrupar por fecha y llave, y sumar la columna 'TOTAL'
energia_por_fecha = df.groupby(['LLAVE NOMBRE', 'FECHA'])['TOTAL'].sum().reset_index()

# Pivotar los datos para tener fechas como columnas y llaves como filas
pivot_df = energia_por_fecha.pivot(index='LLAVE NOMBRE', columns='FECHA', values='TOTAL').fillna(0)

# Aplanar el MultiIndex después de la pivotación
pivot_df.columns = [f'{col.strftime("%Y-%m-%d")}' for col in pivot_df.columns]

# Añadir la columna de nombres de generadoras al DataFrame
pivot_df['LLAVE NOMBRE'] = pivot_df.index

# Convertir el DataFrame a un formato adecuado para el clustering
data = pivot_df.drop(columns=['LLAVE NOMBRE']).values

# Normalizar los datos con MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = scaler.fit_transform(data)

# Normalización específica de series temporales
data_normalized = TimeSeriesScalerMinMax().fit_transform(data_normalized.reshape((data_normalized.shape[0], data_normalized.shape[1], 1)))

# Convertir los datos normalizados en una matriz 2D para linkage
data_reshaped = data_normalized.reshape(data_normalized.shape[0], -1)

# Empezar el cronómetro
start_time = time.time()

# Aplicar el algoritmo de agrupamiento jerárquico aglomerativo (AHC) con el método de ward
linked = linkage(data_reshaped, method='ward')

# Detener el cronómetro
end_time = time.time()

# Calcular el tiempo de ejecución
execution_time = end_time - start_time

# Obtener etiquetas basadas en 10 clusters
labels = fcluster(linked, t=10, criterion='maxclust')

# Añadir las etiquetas de los clústeres al DataFrame
pivot_df['Cluster'] = labels

# Crear un DataFrame con la columna 'LLAVE NOMBRE' y 'TIPO' de df
tipo_df = df[['LLAVE NOMBRE', 'TIPO']].drop_duplicates()

# Obtener la lista de clusters únicos
clusters = pivot_df['Cluster'].unique()

# Crear un diccionario para almacenar los totales y proporciones de tipos por cluster
totales_cluster = {}
porcentaje_tipo_total = {}

# Inicializar un DataFrame para acumular el total de cada tipo
total_tipos_df = tipo_df['TIPO'].value_counts().to_frame(name='Total').reset_index()
total_tipos_df.columns = ['TIPO', 'Total']

# Iterar sobre cada cluster para calcular los totales y proporciones
for cluster in clusters:
    # Filtrar los datos del cluster
    cluster_data = pivot_df[pivot_df['Cluster'] == cluster]
    
    # Obtener la lista de generadoras en el cluster
    generadoras = cluster_data['LLAVE NOMBRE'].unique()
    
    # Obtener el tipo correspondiente para cada generadora
    tipos = tipo_df[tipo_df['LLAVE NOMBRE'].isin(generadoras)]
    
    # Contar las ocurrencias de cada tipo
    tipo_counts = tipos['TIPO'].value_counts()
    
    # Guardar el total de generadoras en el cluster
    total_cluster = tipo_counts.sum()
    totales_cluster[cluster] = total_cluster
    
    # Calcular el porcentaje de cada tipo dentro del cluster y agregar el conteo
    tipo_porcentaje_y_cantidad = tipo_counts.to_frame(name='Cantidad')
    tipo_porcentaje_y_cantidad['Porcentaje'] = (tipo_counts / total_cluster) * 100
    
    # Almacenar los porcentajes y cantidades en el diccionario
    porcentaje_tipo_total[cluster] = tipo_porcentaje_y_cantidad

# Mostrar los resultados totales por tipo y por cluster
print("Proporción total de cada tipo de generadora:")
print(total_tipos_df)
print()

print("Totales, cantidades y porcentajes de cada cluster:")
for cluster, total in totales_cluster.items():
    print(f"Cluster {cluster} - Total generadoras: {total}")
    print(f"Cantidad y porcentaje por tipo en el cluster {cluster}:")
    print(porcentaje_tipo_total[cluster])
    print()  # Línea en blanco para mayor legibilidad

# Generar dendrograma
plt.figure(figsize=(12, 8))
dendrogram(linked, labels=pivot_df['LLAVE NOMBRE'].values, leaf_rotation=90)
plt.title('Dendrograma de Agrupamiento Jerárquico Aglomerativo (Ward)')
plt.xlabel('Generadora')
plt.ylabel('Distancia')
plt.axhline(y=0.5, color='r', linestyle='--')  # Ajusta este valor si es necesario
plt.show()

# Imprimir el tiempo de ejecución
print(f"Tiempo de ejecución: {execution_time:.2f} segundos")


In [ ]:
# Crear un DataFrame con la columna 'LLAVE NOMBRE' y 'REGIÓN' de df
region_df = df[['LLAVE NOMBRE', 'REGIÓN']].drop_duplicates()

# Crear un diccionario para almacenar los totales y proporciones de regiones por cluster
totales_cluster_region = {}
porcentaje_region_total = {}

# Inicializar un DataFrame para acumular el total de cada región
total_regiones_df = region_df['REGIÓN'].value_counts().to_frame(name='Total').reset_index()
total_regiones_df.columns = ['REGIÓN', 'Total']

# Iterar sobre cada cluster para calcular los totales y proporciones
for cluster in clusters:
    # Filtrar los datos del cluster
    cluster_data = sample_df[sample_df['Cluster'] == cluster]
    
    # Obtener la lista de generadoras en el cluster
    generadoras = cluster_data['LLAVE NOMBRE'].unique()
    
    # Obtener la región correspondiente para cada generadora
    regiones = region_df[region_df['LLAVE NOMBRE'].isin(generadoras)]
    
    # Contar las ocurrencias de cada región
    region_counts = regiones['REGIÓN'].value_counts()
    
    # Guardar el total de generadoras en el cluster
    total_cluster = region_counts.sum()
    totales_cluster_region[cluster] = total_cluster
    
    # Calcular el porcentaje de cada región dentro del cluster y agregar el conteo
    region_porcentaje_y_cantidad = region_counts.to_frame(name='Cantidad')
    region_porcentaje_y_cantidad['Porcentaje'] = (region_counts / total_cluster) * 100
    
    # Almacenar los porcentajes y cantidades en el diccionario
    porcentaje_region_total[cluster] = region_porcentaje_y_cantidad

# Mostrar los resultados totales por región y por cluster
print("Proporción total de cada región de generadora:")
print(total_regiones_df)
print()

print("Totales, cantidades y porcentajes de cada cluster por región:")
for cluster, total in totales_cluster_region.items():
    print(f"Cluster {cluster} - Total generadoras: {total}")
    print(f"Cantidad y porcentaje por región en el cluster {cluster}:")
    print(porcentaje_region_total[cluster])
    print()  # Línea en blanco para mayor legibilidad

In [ ]:
# Crear un DataFrame con las columnas 'LLAVE NOMBRE', 'REGIÓN' y 'TIPO' de df, eliminando duplicados
region_tipo_df = df[['LLAVE NOMBRE', 'REGIÓN', 'TIPO']].drop_duplicates()

# Diccionario para almacenar los totales y proporciones de tipos por región y cluster
totales_cluster_region_tipo = {}

# Inicializar un DataFrame para acumular el total de cada región y tipo
total_regiones_df = region_tipo_df.groupby(['REGIÓN', 'TIPO']).size().to_frame(name='Total').reset_index()

# Iterar sobre cada cluster para calcular los totales y proporciones por región y tipo
for cluster in clusters:
    # Filtrar los datos del cluster
    cluster_data = sample_df[sample_df['Cluster'] == cluster]
    
    # Obtener la lista de generadoras en el cluster
    generadoras = cluster_data['LLAVE NOMBRE'].unique()
    
    # Obtener la región y tipo correspondiente para cada generadora
    regiones_tipos = region_tipo_df[region_tipo_df['LLAVE NOMBRE'].isin(generadoras)]
    
    # Contar las ocurrencias de cada combinación de región y tipo
    region_tipo_counts = regiones_tipos.groupby(['REGIÓN', 'TIPO']).size().to_frame(name='Cantidad').reset_index()
    
    # Guardar el total de generadoras en el cluster
    total_cluster = region_tipo_counts['Cantidad'].sum()
    
    # Calcular el porcentaje de cada combinación de región y tipo dentro del cluster
    region_tipo_counts['Porcentaje'] = (region_tipo_counts['Cantidad'] / total_cluster) * 100
    
    # Almacenar los resultados en el diccionario
    totales_cluster_region_tipo[cluster] = region_tipo_counts

# Mostrar los resultados totales por región y tipo en cada cluster
print("Totales, cantidades y porcentajes de cada combinación de región y tipo por cluster:")
for cluster, data in totales_cluster_region_tipo.items():
    print(f"Cluster {cluster} - Total generadoras:")
    print(data)
    print()  # Línea en blanco para mayor legibilidad

# COMPLETE

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from tslearn.preprocessing import TimeSeriesScalerMinMax
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.metrics import pairwise_distances
import time

# Agrupar por fecha y llave, y sumar la columna 'TOTAL'
energia_por_fecha = df.groupby(['LLAVE NOMBRE', 'FECHA'])['TOTAL'].sum().reset_index()

# Pivotar los datos para tener fechas como columnas y llaves como filas
pivot_df = energia_por_fecha.pivot(index='LLAVE NOMBRE', columns='FECHA', values='TOTAL').fillna(0)

# Convertir las columnas de fecha a datetime
pivot_df.columns = pd.to_datetime(pivot_df.columns)

# Añadir la columna de nombres de generadoras al DataFrame
pivot_df['LLAVE NOMBRE'] = pivot_df.index

# Muestreo: seleccionar una fracción aleatoria de los datos (por ejemplo, el 10%)
sample_df = pivot_df.sample(frac=1, random_state=0)  # Ajusta la fracción según sea necesario

# Convertir el DataFrame muestreado a un formato adecuado
data = sample_df.drop(columns=['LLAVE NOMBRE']).values

# Normalización específica de series temporales
data_normalized = TimeSeriesScalerMinMax().fit_transform(data.reshape((data.shape[0], data.shape[1], 1)))

# Convertir los datos normalizados en una matriz 2D para linkage
data_reshaped = data_normalized.reshape(data_normalized.shape[0], -1)

# Empezar el cronómetro
start_time = time.time()

# Aplicar el algoritmo de agrupamiento jerárquico aglomerativo (AHC) con la métrica Complete
linked = linkage(data_reshaped, method='complete')

# Obtener etiquetas basadas en el número de clusters (por ejemplo, 10 clusters)
n_clusters = 10  # Ajusta el número de clusters según sea necesario
labels = fcluster(linked, t=n_clusters, criterion='maxclust')

# Añadir los resultados al DataFrame muestreado
sample_df['Cluster'] = labels

# Reordenar las columnas para mantener 'LLAVE NOMBRE' al principio
sample_df = sample_df[['LLAVE NOMBRE', 'Cluster'] + [col for col in sample_df.columns if col not in ['LLAVE NOMBRE', 'Cluster']]]

# Calcular la matriz de distancias euclidianas
distances = pairwise_distances(data_reshaped)

# Calcular el índice de Silueta usando la métrica euclidiana
silhouette_avg = silhouette_score(distances, labels, metric='euclidean')

# Detener el cronómetro
end_time = time.time()

# Calcular el tiempo de ejecución
execution_time = end_time - start_time

# Imprimir los resultados
print(f"Tiempo de ejecución: {execution_time:.2f} segundos")
print(f"Tiempo de ejecución: {execution_time / 60:.2f} minutos")
print(f"Índice de Silueta: {silhouette_avg:.2f}")

# Limpiar las columnas de `pivot_df` asegurándose de que sean fechas
pivot_df.columns = pd.to_datetime(pivot_df.columns, errors='coerce')

# Eliminar columnas que no se pudieron convertir a fechas
pivot_df = pivot_df.loc[:, pivot_df.columns.notna()]

# Obtener la lista de clusters únicos
clusters = sample_df['Cluster'].unique()

# Crear un DataFrame con la columna 'LLAVE NOMBRE' y 'TIPO' de df
tipo_df = df[['LLAVE NOMBRE', 'TIPO']].drop_duplicates()

# Crear un diccionario para almacenar los totales y proporciones de tipos por cluster
totales_cluster = {}
porcentaje_tipo_total = {}

# Inicializar un DataFrame para acumular el total de cada tipo
total_tipos_df = tipo_df['TIPO'].value_counts().to_frame(name='Total').reset_index()
total_tipos_df.columns = ['TIPO', 'Total']

# Iterar sobre cada cluster para calcular los totales y proporciones
for cluster in clusters:
    # Filtrar los datos del cluster
    cluster_data = sample_df[sample_df['Cluster'] == cluster]
    
    # Obtener la lista de generadoras en el cluster
    generadoras = cluster_data['LLAVE NOMBRE'].unique()
    
    # Obtener el tipo correspondiente para cada generadora
    tipos = tipo_df[tipo_df['LLAVE NOMBRE'].isin(generadoras)]
    
    # Contar las ocurrencias de cada tipo
    tipo_counts = tipos['TIPO'].value_counts()
    
    # Guardar el total de generadoras en el cluster
    total_cluster = tipo_counts.sum()
    totales_cluster[cluster] = total_cluster
    
    # Calcular el porcentaje de cada tipo dentro del cluster y agregar el conteo
    tipo_porcentaje_y_cantidad = tipo_counts.to_frame(name='Cantidad')
    tipo_porcentaje_y_cantidad['Porcentaje'] = (tipo_counts / total_cluster) * 100
    
    # Almacenar los porcentajes y cantidades en el diccionario
    porcentaje_tipo_total[cluster] = tipo_porcentaje_y_cantidad

# Mostrar los resultados totales por tipo y por cluster
print("Proporción total de cada tipo de generadora:")
print(total_tipos_df)
print()

print("Totales, cantidades y porcentajes de cada cluster:")
for cluster, total in totales_cluster.items():
    print(f"Cluster {cluster} - Total generadoras: {total}")
    print(f"Cantidad y porcentaje por tipo en el cluster {cluster}:")
    print(porcentaje_tipo_total[cluster])
    print()  # Línea en blanco para mayor legibilidad

# Calcular las series más cercanas al centroide para cada tipo dentro de cada cluster
for cluster in clusters:
    # Filtrar los datos del cluster
    cluster_data = sample_df[sample_df['Cluster'] == cluster]
    
    # Obtener la lista de generadoras en el cluster
    generadoras = cluster_data['LLAVE NOMBRE'].unique()
    
    # Filtrar las series temporales de las generadoras en el cluster
    cluster_series = pivot_df.loc[generadoras]
    
    # Obtener los tipos de energía en el cluster
    tipos_en_cluster = tipo_df[tipo_df['LLAVE NOMBRE'].isin(generadoras)]
    
    # Iterar sobre cada tipo de energía en el cluster
    for tipo in tipos_en_cluster['TIPO'].unique():
        # Filtrar las series temporales del tipo de energía actual
        series_tipo = cluster_series.loc[tipos_en_cluster[tipos_en_cluster['TIPO'] == tipo]['LLAVE NOMBRE']]
        
        if not series_tipo.empty:
            # Calcular el centroide (media) del tipo de energía
            centroide = series_tipo.mean(axis=0)
            
            # Calcular la distancia entre cada serie temporal y el centroide usando la distancia euclidiana
            distancias = np.array([np.linalg.norm(centroide.values - serie.values) for index, serie in series_tipo.iterrows()]).reshape(-1, 1)
            
            # Encontrar las 3 series más cercanas al centroide
            indices_mas_cercanas = np.argsort(distancias, axis=0)[:3].flatten()
            series_representativas = series_tipo.iloc[indices_mas_cercanas]
            
            # Graficar las 3 series temporales representativas
            fig, ax = plt.subplots(figsize=(12, 6))
            for j, (index, serie) in enumerate(series_representativas.iterrows()):
                ax.plot(pivot_df.columns, serie, label=f'Cluster {cluster} - Tipo {tipo} - Serie {j+1}', linestyle='-', marker='o')
            
            # Configuración del gráfico
            ax.set_title(f'Series Temporales Representativas del Cluster {cluster} - Tipo {tipo}')
            ax.set_xlabel('Fecha')
            ax.set_ylabel('Valor')
            ax.xaxis.set_major_locator(mdates.MonthLocator())
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
            ax.tick_params(axis='x', rotation=45)
            ax.legend(loc='upper right')

            plt.tight_layout()
            plt.show()


In [ ]:
# Crear un DataFrame con la columna 'LLAVE NOMBRE' y 'REGIÓN' de df
region_df = df[['LLAVE NOMBRE', 'REGIÓN']].drop_duplicates()

# Crear un diccionario para almacenar los totales y proporciones de regiones por cluster
totales_cluster_region = {}
porcentaje_region_total = {}

# Inicializar un DataFrame para acumular el total de cada región
total_regiones_df = region_df['REGIÓN'].value_counts().to_frame(name='Total').reset_index()
total_regiones_df.columns = ['REGIÓN', 'Total']

# Iterar sobre cada cluster para calcular los totales y proporciones
for cluster in clusters:
    # Filtrar los datos del cluster
    cluster_data = sample_df[sample_df['Cluster'] == cluster]
    
    # Obtener la lista de generadoras en el cluster
    generadoras = cluster_data['LLAVE NOMBRE'].unique()
    
    # Obtener la región correspondiente para cada generadora
    regiones = region_df[region_df['LLAVE NOMBRE'].isin(generadoras)]
    
    # Contar las ocurrencias de cada región
    region_counts = regiones['REGIÓN'].value_counts()
    
    # Guardar el total de generadoras en el cluster
    total_cluster = region_counts.sum()
    totales_cluster_region[cluster] = total_cluster
    
    # Calcular el porcentaje de cada región dentro del cluster y agregar el conteo
    region_porcentaje_y_cantidad = region_counts.to_frame(name='Cantidad')
    region_porcentaje_y_cantidad['Porcentaje'] = (region_counts / total_cluster) * 100
    
    # Almacenar los porcentajes y cantidades en el diccionario
    porcentaje_region_total[cluster] = region_porcentaje_y_cantidad

# Mostrar los resultados totales por región y por cluster
print("Proporción total de cada región de generadora:")
print(total_regiones_df)
print()

print("Totales, cantidades y porcentajes de cada cluster por región:")
for cluster, total in totales_cluster_region.items():
    print(f"Cluster {cluster} - Total generadoras: {total}")
    print(f"Cantidad y porcentaje por región en el cluster {cluster}:")
    print(porcentaje_region_total[cluster])
    print()  # Línea en blanco para mayor legibilidad

In [ ]:
# Crear un DataFrame con las columnas 'LLAVE NOMBRE', 'REGIÓN' y 'TIPO' de df, eliminando duplicados
region_tipo_df = df[['LLAVE NOMBRE', 'REGIÓN', 'TIPO']].drop_duplicates()

# Diccionario para almacenar los totales y proporciones de tipos por región y cluster
totales_cluster_region_tipo = {}

# Inicializar un DataFrame para acumular el total de cada región y tipo
total_regiones_df = region_tipo_df.groupby(['REGIÓN', 'TIPO']).size().to_frame(name='Total').reset_index()

# Iterar sobre cada cluster para calcular los totales y proporciones por región y tipo
for cluster in clusters:
    # Filtrar los datos del cluster
    cluster_data = sample_df[sample_df['Cluster'] == cluster]
    
    # Obtener la lista de generadoras en el cluster
    generadoras = cluster_data['LLAVE NOMBRE'].unique()
    
    # Obtener la región y tipo correspondiente para cada generadora
    regiones_tipos = region_tipo_df[region_tipo_df['LLAVE NOMBRE'].isin(generadoras)]
    
    # Contar las ocurrencias de cada combinación de región y tipo
    region_tipo_counts = regiones_tipos.groupby(['REGIÓN', 'TIPO']).size().to_frame(name='Cantidad').reset_index()
    
    # Guardar el total de generadoras en el cluster
    total_cluster = region_tipo_counts['Cantidad'].sum()
    
    # Calcular el porcentaje de cada combinación de región y tipo dentro del cluster
    region_tipo_counts['Porcentaje'] = (region_tipo_counts['Cantidad'] / total_cluster) * 100
    
    # Almacenar los resultados en el diccionario
    totales_cluster_region_tipo[cluster] = region_tipo_counts

# Mostrar los resultados totales por región y tipo en cada cluster
print("Totales, cantidades y porcentajes de cada combinación de región y tipo por cluster:")
for cluster, data in totales_cluster_region_tipo.items():
    print(f"Cluster {cluster} - Total generadoras:")
    print(data)
    print()  # Línea en blanco para mayor legibilidad